In [ ]:
import os
os.cpu_count()

2

In [ ]:
# install java libs and spark.
!apt update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# set environment variables for java and spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"


In [ ]:
# add pyspark to sys.path
import findspark
findspark.init()

In [ ]:
! java --version

openjdk 11.0.11 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('assignment_spark').getOrCreate()

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sc = spark.sparkContext
rdd = sc.textFile('/content/drive/MyDrive/AirlineInfo/files/*.csv.bz2')

In [ ]:
rdd.take(2)

['Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay',
 '1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA']

In [ ]:
sparkDF = rdd.map(lambda x: str(x)).map(lambda w: w.split(',')).toDF()

In [ ]:
# Let's checkout out dataset by printing first 10 rows
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT * from DATA limit 10;").show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  _1|   _2|        _3|       _4|     _5|        _6|     _7|        _8|           _9|      _10|    _11|              _12|           _13|    _14|     _15|     _16|   _17| _18|     _19|   _20|    _21|      _22|             _23|     _24|         _25|         _26|     _27|          _28|              _29|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum

In [ ]:
# Find the # of flights each airline made so far from 1987 until recent
# Here we will count the total no. of flights from each airline. Since we including all the years, we don't need to filter for years.
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._9, count(data._10) from DATA group by data._9 order by data._9;").show()

+------+----------+
|    _9|count(_10)|
+------+----------+
|    9E|    521059|
|    AA|  14984647|
|    AQ|    154381|
|    AS|   2878021|
|    B6|    811341|
|    CO|   8145788|
|    DH|    693047|
|    DL|  16547870|
|    EA|    919785|
|    EV|   1697172|
|    F9|    336958|
|    FL|   1265138|
|    HA|    274265|
|    HP|   3636682|
|ML (1)|     70622|
|    MQ|   3954895|
|    NW|  10292627|
|    OH|   1464176|
|    OO|   3090853|
|PA (1)|    316167|
+------+----------+
only showing top 20 rows



In the above table, the first column shows all the unique carrier code from each airline and the second column shows the total no. of flights for that airline 

In [ ]:
# Find the mean departure delay per origination airport.
# What is the average departure delay from each airport
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._17, avg(data._16) from DATA group by data._17 order by data._17;").show()

+---+------------------------+
|_17|avg(CAST(_16 AS DOUBLE))|
+---+------------------------+
|ABE|      7.9272827780290465|
|ABI|      3.8167625695689087|
|ABQ|      6.7483899198440636|
|ABY|       9.886588658865886|
|ACK|      27.383211678832115|
|ACT|      1.5437979383457912|
|ACV|      11.044615183131794|
|ACY|       5.264685395867193|
|ADK|      22.731977818853974|
|ADQ|       8.925332778702163|
|AEX|       6.921160836660509|
|AGS|       7.590694188191882|
|AKN|       10.10936420179682|
|ALB|        6.78996656683151|
|ALO|       5.477433004231312|
|AMA|       5.964805154955508|
|ANC|       7.963208352025641|
|ANI|       6.555555555555555|
|APF|       7.952959028831563|
|ASE|      12.586657443701483|
+---+------------------------+
only showing top 20 rows



Here in the above table, we have origin airport code on the left and the average departure delay on the right for that airport

In [ ]:
# What is the average departure delay from each airport?
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._17, avg(data._16) from DATA group by data._17 order by data._17;").show()

+---+------------------------+
|_17|avg(CAST(_16 AS DOUBLE))|
+---+------------------------+
|ABE|      7.9272827780290465|
|ABI|      3.8167625695689087|
|ABQ|      6.7483899198440636|
|ABY|       9.886588658865886|
|ACK|      27.383211678832115|
|ACT|      1.5437979383457912|
|ACV|      11.044615183131794|
|ACY|       5.264685395867193|
|ADK|      22.731977818853974|
|ADQ|       8.925332778702163|
|AEX|       6.921160836660509|
|AGS|       7.590694188191882|
|AKN|       10.10936420179682|
|ALB|        6.78996656683151|
|ALO|       5.477433004231312|
|AMA|       5.964805154955508|
|ANC|       7.963208352025641|
|ANI|       6.555555555555555|
|APF|       7.952959028831563|
|ASE|      12.586657443701483|
+---+------------------------+
only showing top 20 rows



Here in the above table, we have airport code on the left and the average departure delay on the right for that airpor

In [ ]:
# What day the delays are the worst?
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._3, max(data._16) from DATA where data._16 <> 'NA' group by data._3 order by data._3 desc;").show()

+---+--------+
| _3|max(_16)|
+---+--------+
|  1|     994|
| 10|     997|
| 11|     991|
| 12|     999|
| 13|     999|
| 14|     997|
| 15|     991|
| 16|     998|
| 17|     996|
| 18|     998|
| 19|     999|
|  2|     999|
| 20|     997|
| 21|     999|
| 22|     998|
| 23|     995|
| 24|     990|
| 25|     996|
| 26|     998|
| 27|     996|
+---+--------+
only showing top 20 rows



Here in the above table we have day of the month on left and the max no. of delays on the right for that day

In [ ]:
# Which day of the week is the most of the flights cancelled?
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._4, sum(data._22)  from DATA where data._22<>'NA' group by data._4 order by data._4;").show()

+---------+------------------------+
|       _4|sum(CAST(_22 AS DOUBLE))|
+---------+------------------------+
|        1|                344508.0|
|        2|                376554.0|
|        3|                363496.0|
|        4|                356908.0|
|        5|                340886.0|
|        6|                263131.0|
|        7|                257841.0|
|DayOfWeek|                    null|
+---------+------------------------+



From the table above, looks like most flights were canceled on the 2nd day of the week

In [ ]:
# Which day of the month is the most of the flights cancelled?
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._3, sum(data._22)  from DATA where data._22<>'NA' group by data._3 order by data._3 desc;").show()

+----------+------------------------+
|        _3|sum(CAST(_22 AS DOUBLE))|
+----------+------------------------+
|DayofMonth|                    null|
|         9|                 77471.0|
|         8|                 79673.0|
|         7|                 74514.0|
|         6|                 80456.0|
|         5|                 75030.0|
|         4|                 74104.0|
|        31|                 42031.0|
|        30|                 62031.0|
|         3|                 67814.0|
|        29|                 59860.0|
|        28|                 65426.0|
|        27|                 66611.0|
|        26|                 70084.0|
|        25|                 72709.0|
|        24|                 63621.0|
|        23|                 71394.0|
|        22|                 71069.0|
|        21|                 67306.0|
|        20|                 65473.0|
+----------+------------------------+
only showing top 20 rows



So from the above table, we can say that most flights were canceled on the 6th day of the month

In [ ]:
# Find the on-time (ArrTime - CRSArrTime <= 0) performance for each unique carrier.
sparkDF.createOrReplaceTempView("DATA")
spark.sql("SELECT data._9, sum(data._7 - data._8) from DATA where data._8<=0 group by data._9 order by data._9;").show()

+---+----------------------------------------------+
| _9|sum((CAST(_7 AS DOUBLE) - CAST(_8 AS DOUBLE)))|
+---+----------------------------------------------+
| AA|                                  7.89070988E8|
| DH|                                        2359.0|
| EV|                                        2400.0|
| FL|                                       16821.0|
| HA|                                        2334.0|
| OO|                                       75791.0|
| UA|                                  1.47916537E8|
| XE|                                          null|
| YV|                                          null|
+---+----------------------------------------------+



Here in the above table we have unique carrier code on the left and on time performance on the right. The carrier AA appears to have the best on time performance